<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Получение-и-обзор-данных" data-toc-modified-id="Получение-и-обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Получение и обзор данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков-и-некорректных-значений" data-toc-modified-id="Обработка-пропусков-и-некорректных-значений-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков и некорректных значений</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация-и-категоризация-данных" data-toc-modified-id="Лемматизация-и-категоризация-данных-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация и категоризация данных</a></span></li></ul></li><li><span><a href="#Проверка-гипотез-исследования" data-toc-modified-id="Проверка-гипотез-исследования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка гипотез исследования</a></span></li><li><span><a href="#Общий-вывод-исследования" data-toc-modified-id="Общий-вывод-исследования-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод исследования</a></span></li></ul></div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Рабочие гипотезы**

- наличие детей влияет на возврат кредита в срок
- возврат кредита в срок зависит от семейного положения заемщика
- возврат кредита в срок зависит от уровня дохода заемщика
- различные цели кредита влияют на возврат его в срок

## Получение и обзор данных

In [1]:
# импортируем библиотеку pandas
import pandas as pd
# импортируем библиотеку для лемматизации pymystem3
from pymystem3 import Mystem
m = Mystem()

Прочитаем файл *data.csv* и сохраним его в переменной `data`:

In [2]:
# чтение файла с данными и сохранение в переменной 'data'
data = pd.read_csv('data.csv')

Получим первые десять строк таблицы:

In [3]:
# получение первых 10 строк таблицы data методом head()
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о данных в таблице:

In [4]:
# получение общей информации о таблице data методом info()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов с типами данных - целочисленный `int64`, вещественный `float64` и строковый объект `object`.

Опишем информацию в столбцах таблицы `data`:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Определим видимые артефакты:
- количество значений в столбцах `days_employed` и `total_income` отличается от общего количества записей в таблице, что говорит о пропущенных значениях в данных столбцах
- значения полей в столбце `education` не приведены к нижнему регистру
- в столбце `days_employed` присутствуют некорректные данные - довольно большие и отрицательные значения 
- словоформы значений в столбце `purpose` не приведены к лемме (начальной, словарной форме слова), что помешает категоризации данных

Также в исходном датасете могут присутствовать скрытые аномалии в некоторых столбцах. Как и видимые, их предстоить выявить и обработать на следующем этапе исследования.

**Вывод**

Каждая запись в таблице содержит информацию о платежеспособности клиентов банка: семейном, социальном и трудовом статусе, а также о целях кредита и задолженности по кредитным обязательствам. Входные данные содержат видимые проблемы: пропущенные и некорректные значения, возможные дубликаты данных, словоформы, не приведенные к лемме. Наиболее важные столбцы с точки зрения проверки гипотез - количество детей в семье (`children`), семейное положение (`family_status`), ежемесячный доход (`total_income`) и цель получения кредита (`purpose`).

## Предобработка данных

### Обработка пропусков и некорректных значений

Узнаем, сколько значений явно пропущено в столбцах таблицы `data`:

In [5]:
# определение пропущенных значений в таблице data
# методом isna() и их количества методом sum()
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах `days_employed` (общий трудовой стаж в днях) и `total_income` (ежемесячный доход) пропущено по 2174 записи. 

Рассмотрим столбец `days_employed` подробнее:

In [6]:
# получение первых 10 строк столбца 'days_employed' методом head()
data['days_employed'].head(10)

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
5      -926.185831
6     -2879.202052
7      -152.779569
8     -6929.865299
9     -2188.756445
Name: days_employed, dtype: float64

Присутствуют отрицательные значения, а также выбросы в виде несопоставимых с логикой данных в столбце больших чисел. Представленные артефакты могли появиться вследствие проблем с выгрузкой данных - строки могли быть получены из разных систем с разными видами хранения трудового стажа.

Вероятно, отрицательные числа - это разница между датой начала трудового стажа и датой его окончания (или датой выгрузки данных). Можем их исправить, взяв по модулю.

In [7]:
# получение значений столбца 'days_employed' по модулю
data['days_employed'] = abs(data['days_employed'])

# проверка - получение первых 10 строк методом head()
data['days_employed'].head(10)

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
5       926.185831
6      2879.202052
7       152.779569
8      6929.865299
9      2188.756445
Name: days_employed, dtype: float64

Большие значения могут быть обусловлены хранением числовых данных не в днях, а в часах. Поэтому часть строк придется делить на 24, а часть оставить без изменений. 

Предположим, что максимальный стаж работы равен 60 годам. С помощью лямбда-функции поделим значения столбца на 365, и строки с числами меньше 60 будут стажем в годах (их оставим без изменения), больше 60 - в часах. Значения в часах поделим на 24 и заменим строки:

In [8]:
# применение лямбда-функции к столбцу 'days_employed'
data['days_employed'] = data['days_employed'].apply(lambda val: val if val / 365 <= 60 else val / 24)

# проверка - получение первых 10 строк таблицы data методом head()
data['days_employed'].head(10)

0     8437.673028
1     4024.803754
2     5623.422610
3     4124.747207
4    14177.753002
5      926.185831
6     2879.202052
7      152.779569
8     6929.865299
9     2188.756445
Name: days_employed, dtype: float64

Значения приведены к адекватному виду.

Возможно, что клиенты, у которых отсутствует информация о трудовом стаже и доходе, никогда официально не работали и на данный момент также нетрудоустроены. Проверим это:

In [9]:
# сохранение в переменную 'data_pass' строк, в которых отсутствуют значения  
# для столбцов 'days_employed' и (&) 'total_income' методом isna()
data_pass = data[data['days_employed'].isna() & data['total_income'].isna()]

# проверка - получение размера таблицы data_pass методом shape
data_pass.shape

(2174, 12)

Определим тип занятости (`income_type`) клиентов в данной выборке:

In [10]:
# получение уникальных значений по столбцу 'income_type'  
# и их количества в выборке методом value_counts()
data_pass['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Как видно, у всех клиентов в данной выборке присутствует определенный тип занятости, значит должен иметься трудовой стаж и ежемесячный доход. Предположение о безработице клиентов с отсутствующей информацией о доходе не подтвердилось.

Возможной причиной пропусков является проблема с записью данных, так как в банке при подаче заявки на кредит не представляется возможным скрыть ежемесячный доход и трудовой стаж.

Заполним пропуски в полях `days_employed` (трудовой стаж в днях) и `total_income` (ежемесячный доход) медианными значениями стажа и дохода каждой категории клиентов.

Для начала создадим функцию `create_median_income_dict`, которая будет возвращать словарь, где ключ - категория занятости, значение - медиана числовых значений по категории занятости в переданном функции столбце:

In [11]:
# создание функции 'create_median_income_dict()' с аргументом 'columns':
def create_median_income_dict(columns):
    ''' 
    Возвращает словарь, где ключ - категория занятости, значение - медиана 
    числовых значений по категории занятости в переданном функции столбце.
    ''' 
    median_dict = data.sort_values(by=columns)\
                          .groupby('income_type')[columns].median().to_dict()
    return median_dict

Вызовем функцию для столбца `days_employed`. Полученный словарь с ключом - категорией занятости, значением - медианой трудового стажа сохраним в переменной `median_employed`:

In [12]:
# создание словаря медианных значений трудового стажа 'median_employed'
median_employed = create_median_income_dict('days_employed')

#вывод переменной 'median_employed'
median_employed

{'безработный': 15267.235531008522,
 'в декрете': 3296.7599620220594,
 'госслужащий': 2689.3683533043886,
 'компаньон': 1547.3822226779334,
 'пенсионер': 15217.221094405466,
 'предприниматель': 520.8480834953765,
 'сотрудник': 1574.2028211070851,
 'студент': 578.7515535382181}

Аналогично для столбца `total_income` и словаря `median_income`. Ключ - категория занятости, значение - медиана дохода: 

In [13]:
# создание словаря медианных значений дохода 'median_income'
median_income = create_median_income_dict('total_income')

#вывод переменной 'median_income'
median_income

{'безработный': 131339.7516762103,
 'в декрете': 53829.13072905995,
 'госслужащий': 150447.9352830068,
 'компаньон': 172357.95096577113,
 'пенсионер': 118514.48641164352,
 'предприниматель': 499163.1449470857,
 'сотрудник': 142594.39684740017,
 'студент': 98201.62531401133}

Заменим пропуски в столбце `days_employed` таблицы `data`. Убедимся, что их больше не осталось:

In [14]:
# замена пропущенных значений столбца 'days_employed' методом map()
# с использованием словаря 'median_employed' в качестве значений для подстановки
data.loc[data['days_employed'].isna(), 'days_employed'] = data['income_type'].map(median_employed)

# проверка - определение пропущенных значений в столбце
# 'days_employed' методом isna() и их количества методом sum()
data['days_employed'].isna().sum()

0

То же самое для столбца `total_income`:

In [15]:
# замена пропущенных значений столбца 'total_income' методом map()
# с использованием словаря 'median_income' в качестве значений для подстановки
data.loc[data['total_income'].isna(), 'total_income'] = data['income_type'].map(median_income)

# проверка - определение пропущенных значений в столбце
# 'total_income' методом isna() и их количества методом sum()
data['total_income'].isna().sum()

0

Продолжим искать пропуски в данных. Попробуем обнаружить скрытые несостыковки в числовых столбцах - они могли быть заполнены ранее некорректными дефолтными значениями или возникнуть из-за технических проблем.

Проверим столбец `dob_years` — возраст клиента в годах:

In [16]:
# получение уникальных значений по столбцу 'dob_years'  
# и их количества в выборке методом value_counts()
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Видим возраст клиента со значением `0` - это явная ошибка. Исправим - заполним пропуски медианным значением по категориям занятости.

In [17]:
# создание словаря медианных значений дохода 'median_age'
median_age = create_median_income_dict('dob_years')

#вывод перемнной 'median_income'
median_age

{'безработный': 38.0,
 'в декрете': 39.0,
 'госслужащий': 40.0,
 'компаньон': 39.0,
 'пенсионер': 60.0,
 'предприниматель': 42.5,
 'сотрудник': 39.0,
 'студент': 22.0}

In [18]:
# замена ошибочных значений столбца 'dob_years' методом map()
# с использованием словаря 'median_age' в качестве значений для подстановки
data.loc[data['dob_years'] == 0, 'dob_years'] = data['income_type'].map(median_age)

# проверка - получение количества строк таблицы data с 
# нулевым значением столбца 'dob_years' методом count()
data[data['dob_years'] == 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Проверим также на скрытые пропуски столбец `total_income`:

In [19]:
# получение количества строк таблицы data с нулевым
# значением столбца 'total_income' = методом count()
data[data['total_income'] == 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

И поле `days_employed`:

In [20]:
# получение количества строк таблицы data с нулевым
# значением столбца 'days_employed' = методом count()
data[data['days_employed'] == 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Нулевые и `NaN` значения в числовых данных исправлены. Но это еще не значит, что все значения, представленные в столбцах, корректны.

Проверим на отрицательные значения столбец `total_income`:

In [21]:
# получение количества строк таблицы data со значением
# меньше нуля столбца 'total_income' = методом count()
data[data['total_income'] < 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Отрицательные значения отсутствуют. Также проверим на корректность поле `children`:

In [22]:
# получение уникальных значений по столбцу 'children'  
# и их количества в выборке методом value_counts()
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Наблюдаем выбросы в виде отрицательного и слишком большого количества детей. Данные артефакты могли появиться вследствие ранней передобработки данных в этом столбце, так как `-1` - распространенная "заглушка" для нулевых значений, значение `20` также похоже на дефолтное.

Поэтому предположим, что в случае с `-1` и `20` детьми подразумевалось `0`. 

Произведем замену значений:

In [23]:
# замена аномальных значений в столбце 'children' методом 'replace()'
data['children'] = data['children'].replace({-1: 0, 20: 0})

# проверка - получение уникальных значений по столбцу 'children'  
# и их количества в выборке методом value_counts()
data['children'].value_counts()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

В нашем случае имеет место *MAR (Missing At Random)*  - дословно "отсутствует случайно". 
При *MAR* пропуск может быть заполнен на основе имеющейся в наборе данных информации. В данном разделе заполнение явных и скрытых пропусков проводилось медианными значениями по категориям для сохранения репрезентативности числовых характеристик выборки.

### Замена типа данных

После замены нулевых значений возраста медианными столбец `dob_years` приобрел вещественный тип данных:

In [24]:
# получение типа столбца 'dob_years' методом dtypes()
data['dob_years'].dtypes

dtype('float64')

Данный тип хранения значений возраста не совсем корректен и в последующем будет усложнять процесс анализа показателей. Вернем столбцу первоначальный тип:

In [25]:
# приведение столбца 'dob_years' методом astype()
# с аргументом 'int' к целочисленному типу
data['dob_years'] = data['dob_years'].astype(int)

Значения в полях `total_income` и `days_employed` также имеют вещественный тип `float64`. Для удобства и возрастания скорости расчетов приведем к целочисленному `int`:

In [26]:
# приведение столбца 'total_income' методом astype()
# с аргументом 'int' к целочисленному типу
data['total_income'] = data['total_income'].astype(int)

Аналогичные действия проведем и со столбцом `days_employed`:

In [27]:
# приведение столбца 'days_employed' методом astype()
# с аргументом 'int' к целочисленному типу
data['days_employed'] = data['days_employed'].astype(int)

Посмотрим на получившийся результат:

In [28]:
# получение первых 5 строк таблицы data методом head()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Приведение вещественного типа `float64` к целочисленному `int` в нашем случае исключает появление пропусков значений  `NaN` в столбце, ускоряет расчеты и придает лаконичный вид данным.

### Обработка дубликатов

Определим количество дубликатов в таблице `data` и сохраним это значение в переменной `initial_duplicated_sum`:

In [29]:
# определение дубликатов в таблице data методом duplicated() и их количества
# методом sum() с сохранением в переменной 'initial_duplicated_sum'
initial_duplicated_sum = data.duplicated().sum()

# вывод переменной 'initial_duplicated_sum'
initial_duplicated_sum

54

Как было зачемено ранее, значения полей в столбце `education` не приведены к нижнему регистру. Подсчитаем количество уникальных значений в этом поле:

In [30]:
# получение уникальных значений по столбцу 'education'  
# и их количества в выборке методом value_counts()
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Действительно, значения в разном регистре дублируют друг друга. Избавимся от дубликатов приведением строк к нижнему регистру:

In [31]:
# приведение значений в поле 'education' к нижнему
# регистру методом str с атрибутом lower()
data['education'] = data['education'].str.lower()

# проверка - получение уникальных значений по столбцу   
# 'education' и их количества в выборке методом value_counts()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Уникальных значений стало меньше - видны четкие категории по виду образования.

Проверим столбец `family_status`:

In [32]:
# получение уникальных значений по столбцу 'family_status'  
# и их количества в выборке методом value_counts()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Различия в категориальных переменных не видны, но значение поля `Не женат / не замужем` написано с заглавной буквы. Для лаконичности приведем все значения к нижнему регистру:

In [33]:
# приведение значений в поле 'family_status' к нижнему
# регистру методом str с атрибутом lower()
data['family_status'] = data['family_status'].str.lower()

# проверка - получение уникальных значений по столбцу   
# 'education' и их количества в выборке методом value_counts()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Значения столбца `income_type` проверялись на уникальность ранее, при определении категорий по типу занятости - артефактов выявлено не было. После проведенных манипуляций еще раз определим количество дубликатов в таблице `data` и сохраним это значение в переменной `later_duplicated_sum`:

In [34]:
# определение дубликатов в таблице data методом duplicated() и их количества
# методом sum() с сохранением в переменной 'later_duplicated_sum'
later_duplicated_sum = data.duplicated().sum()

# вывод переменной 'later_duplicated_sum'
later_duplicated_sum

71

Сравним переменные `initial_duplicated_sum` и `later_duplicated_sum`:

In [35]:
# сравнение переменных 'initial_duplicated_sum' и 'later_duplicated_sum' 
# и вывод заключения на экран с помощью конструкции if-elif-else 
if initial_duplicated_sum < later_duplicated_sum:
    print('Количество дубликатов после приведения строк к нижнему регистру увеличилось на',\
         later_duplicated_sum - initial_duplicated_sum)
elif initial_duplicated_sum > later_duplicated_sum:
    print('Количество дубликатов после приведения строк к нижнему регистру уменьшилось на',\
         initial_duplicated_sum - later_duplicated_sum)
else:
    print('Количество дубликатов после приведения строк к нижнему регистру не изменилось') 

Количество дубликатов после приведения строк к нижнему регистру увеличилось на 17


Удалим дубликаты в таблице `data`:

In [36]:
# удаление дубликатов в таблице data методом drop_duplicates() и сброс индексов методом
# reset_index() и значением drop=True (чтобы не создавать столбец со старыми значениями индексов)
data = data.drop_duplicates().reset_index(drop=True)

**Вывод**

Возможных причин появления дубликатов в столбце `education` две: человеческий фактор при внесении заявки на кредит в форму банка, а также возникновение технических проблем - данные могли быть получены из разных систем с разными регистрами представления категориальных переменных. Появление же полностью одинаковых полей в таблице `data` возможно из-за дублирования записей в процессе выгрузки данных. Поиск дубликатов должен производится как специальными методами, так и вручную с учетом регистра, их присутствие в таблице может привести к некорректным результатам исследования.

### Лемматизация и категоризация данных

Как было зачемено ранее, не все словоформы значений в столбце `purpose` (цель получения кредита) приведены к лемме (начальной, словарной форме). Игнорирование данного факта помешает категоризации данных - проверка вхождения опредленных подстрок в строку даст некорректный результат. Убедимся в этом - подсчитаем количество уникальных значений поля `purpose`:

In [37]:
# получение уникальных значений по столбцу 'purpose'  
# и их количества в выборке методом value_counts()
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Прослеживаются четыре основные категории целей получения кредита:
1. недвижимость
2. автомобиль
3. образование
4. свадьба

Лемматизируем значения в столбце `purpose` и обозначим категории в новом столбце `type_purpose`, а их id в `type_purpose_id`. Для этого ранее была импортирована библиотека `pymystem3`, которая с помощью метода `lemmatize()` выдает список лемматизированных слов.

Создадим функцию `create_purpose_category_and_id`, принимающую на вход записи столбца `purpose` с возвратом названий и id категорий на выходе:

In [38]:
# создание функции 'create_purpose_category_and_id()' с аргументом 'value'
def create_purpose_category_and_id(value):
    ''' 
    Возвращает категорию цели кредита и ее id по лемматизированным значениям 'value'
    (вхождении названия категории в список лемм 'purpose_lemm', сформированным методом
    lemmatize() для каждой записи):
        - 'недвижимость, 0' при нахождении слов 'недвижимость' или 'жилье'
        - 'автомобиль, 1' при нахождении слова 'автомобиль'
        - 'образование, 2' при нахождении слова 'образование'
        - 'свадьба, 3' при нахождении слова 'свадьба' 
        - 'прочие цели, 4' во всех остальных случаях
    '''
    purpose_lemm = m.lemmatize(value)
    if 'недвижимость' in purpose_lemm:
        return pd.Series(['недвижимость', 0])
    if 'жилье' in purpose_lemm:
        return pd.Series(['недвижимость', 0])
    if 'автомобиль' in purpose_lemm:
        return pd.Series(['автомобиль', 1])
    if 'образование' in purpose_lemm:
        return pd.Series(['образование', 2])  
    if 'свадьба' in purpose_lemm:
        return pd.Series(['свадьба', 3])
    return pd.Series(['иные цели', 4])

Применим данную функцию к таблице `data` - создадим новые столбцы `purpose_type` (тип цели кредита) и `purpose_type_id` (id категории цели кредита):

In [39]:
# создание столбцов 'purpose_type' и 'purpose_type_id' в таблице data методом
# apply() с применением функции create_category_purpose_and_id() к столбцу 'purpose'
data[['purpose_type', 'purpose_type_id']] = data['purpose'].apply(create_purpose_category_and_id)

# проверка - получение первых 5 строк таблицы data_cut методом head()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type,purpose_type_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,2
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,3


Категоризируем клиентов по уровню дохода. Для этого обозначим сами уровни:
* нижний квартиль (первые 25% значений уровня дохода) - малоимущий
* интерквартильный размах (центральные 50% значений) - средний класс
* верхняя квартиль (верхние 25%) - богатый

Создадим функцию `create_total_income_type_and_id`, принимающую на вход значения дохода с возвратом названий и id категорий на выходе:

In [40]:
# создание функции 'create_total_income_type_and_id()' с аргументом 'value'
def create_total_income_type_and_id(value):
    ''' 
    Возвращает категорию клиента по уровню дохода и ее id сравнивая значения 'value'
    со значениями квартилей столбца 'total_income', округленных до ближайшего целого
    параметром interpolation='nearest':
        - 'малоимущий, 2' при доходе меньше первого квартиля
        - 'средний класс, 1' при доходе в промежутке интерквартильного размаха
        - 'богатый, 0' при доходе выше третьего квартиля
    '''
    lower_quantile = data['total_income'].quantile(0.25, interpolation='nearest')
    upper_quantile = data['total_income'].quantile(0.75, interpolation='nearest')
    if value <= lower_quantile:
        return pd.Series(['малоимущий', 2])
    if value >= upper_quantile:
        return pd.Series(['богатый', 0])
    return pd.Series(['средний класс', 1])

Применим данную функцию к таблице `data` - создадим новые столбцы `total_income_type` (категория дохода) и `total_income_type_id` (id категории дохода):

In [41]:
# создание столбцов 'total_income_type' и 'total_income_type_id' в таблице data методом
# apply() с применением функции create_total_income_type_and_id() к столбцу 'total_income'
data[['total_income_type', 'total_income_type_id']] = data['total_income']\
                                                          .apply(create_total_income_type_and_id)

Поместим полученные столбцы после `total_income`:

In [42]:
# размещение столбца 'total_income_type' в таблице data после столбца
# 'total_income' методами insert() (вставка) и pop() (извлечение)
data.insert(data.columns.get_loc('total_income')+1,'total_income_type',\
            data.pop('total_income_type'))

# размещение столбца 'total_income_type_id' в таблице data после столбца
# 'total_income_type' методами insert() (вставка) и pop() (извлечение)
data.insert(data.columns.get_loc('total_income_type')+1,'total_income_type_id',\
            data.pop('total_income_type_id'))

# проверка - получение первых 5 строк таблицы data_cut методом head()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,total_income_type,total_income_type_id,purpose,purpose_type,purpose_type_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,богатый,0,покупка жилья,недвижимость,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,средний класс,1,приобретение автомобиля,автомобиль,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,средний класс,1,покупка жилья,недвижимость,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,богатый,0,дополнительное образование,образование,2
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,средний класс,1,сыграть свадьбу,свадьба,3


Классифицируем по типу значения в столбце `income_type`. Для этого создадим словарь `income_type_dict_insert`, где ключи - типы занятости столбца `income_type`, значения - id категорий по типам занятости:

In [43]:
# создание словаря 'income_type_dict_insert'
income_type_dict_insert = {'сотрудник': 0,
                           'пенсионер': 1,
                           'компаньон': 2,
                           'госслужащий': 3,
                           'безработный': 4,
                           'предприниматель': 5,
                           'студент': 6,
                           'в декрете': 7}

В таблице `data` создадим новый столбец `income_type_id` (id типа занятости) и расположим его после `income_type`:

In [44]:
# создание столбца 'income_type_id' в таблице data методом map() с использованием
# словаря 'income_type_dict_insert' в качестве значений для подстановки
data['income_type_id'] = data['income_type'].map(income_type_dict_insert)

# размещение столбца 'income_type_id' в таблице data после столбца
# 'income_type' методами insert() (вставка) и pop() (извлечение)
data.insert(data.columns.get_loc('income_type')+1,'income_type_id',\
            data.pop('income_type_id'))

# проверка - получение первых 5 строк таблицы data методом head()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,income_type_id,debt,total_income,total_income_type,total_income_type_id,purpose,purpose_type,purpose_type_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,0,253875,богатый,0,покупка жилья,недвижимость,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,0,112080,средний класс,1,приобретение автомобиля,автомобиль,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,0,145885,средний класс,1,покупка жилья,недвижимость,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,0,267628,богатый,0,дополнительное образование,образование,2
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,1,0,158616,средний класс,1,сыграть свадьбу,свадьба,3


Также классифицируем по типу значения в столбце `children`. Для этого создадим создадим функцию `create_children_type_and_id`, принимающую на вход записи столбца `children` с возвратом названий и id категорий на выходе:

In [45]:
# создание функции 'create_children_type_and_id()' с аргументом 'value'
def create_children_type_and_id(value):
    ''' 
    Возвращает категорию клиента по количеству детей и ее id сравнивая 
    значения 'value' по условию:
        - 'бездетный, 0' при отсутствии детей
        - 'малодетный, 1' при наличии одного-двух детей
        - 'многодетный, 2' при наличии более 3 детей
    '''
    if value == 0:
        return pd.Series(['бездетный', 1])
    if value >= 3:
        return pd.Series(['многодетный', 2])
    return pd.Series(['малодетный', 0])

В таблице `data` создадим новые поля `children_type` (категория детности) и `children_type_id` (id категории детности):

In [46]:
# создание столбцов 'children_type' и 'children_type_id' в таблице data методом apply()
# с применением функции create_children_type_and_id() к столбцу 'children'
data[['children_type', 'children_type_id']] = data['children'].apply(create_children_type_and_id)

Поместим полученные столбцы после `children`:

In [47]:
# размещение столбца 'children_type' в таблице data после столбца
# 'children' методами insert() (вставка) и pop() (извлечение)
data.insert(data.columns.get_loc('children')+1,'children_type',\
            data.pop('children_type'))

# размещение столбца 'tchildren_type_id' в таблице data после столбца
# 'children_type' методами insert() (вставка) и pop() (извлечение)
data.insert(data.columns.get_loc('children_type')+1,'children_type_id',\
            data.pop('children_type_id'))

# проверка - получение первых 5 строк таблицы data_cut методом head()
data.head()

,children,children_type,children_type_id,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,income_type_id,debt,total_income,total_income_type,total_income_type_id,purpose,purpose_type,purpose_type_id
0,1,малодетный,0,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,0,253875,богатый,0,покупка жилья,недвижимость,0
1,1,малодетный,0,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,0,112080,средний класс,1,приобретение автомобиля,автомобиль,1
2,0,бездетный,1,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,0,145885,средний класс,1,покупка жилья,недвижимость,0
3,3,многодетный,2,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,0,267628,богатый,0,дополнительное образование,образование,2
4,0,бездетный,1,14177,53,среднее,1,гражданский брак,1,F,пенсионер,1,0,158616,средний класс,1,сыграть свадьбу,свадьба,3


Напишем функцию `create_category_dict` для создания "словарей" категоризированных данных в столбцах. Она будет возвращать датафрейм с "ключом" (категорией) и "значением" (id категориии):

In [48]:
# создание функции 'create_category_dict()' с аргументами 'key' и 'value'
def create_category_dict(key, value):
    ''' 
    Возвращает датафрейм с "ключом" (категорией) - 'key' 
    и "значением" (id категориии) - 'value'
    ''' 
    category_dict = data[[key, value]]\
                        .drop_duplicates().reset_index(drop=True)

    return category_dict

Вызовем функцию для столбцов `education` и `education_id`. Получим словарь `edu_dict`:

In [49]:
# вызов функции 'create_category_dict()' для столбцов 'education' и 'education_id'
edu_dict = create_category_dict('education', 'education_id')

# вывод переменной 'edu_dict'
edu_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


То же самое для полей `family_status` и `family_status_id` с переменной `family_stat_dict`:

In [50]:
# вызов функции 'create_category_dict()' для столбцов 'family_status' и 'family_status_id'
family_stat_dict = create_category_dict('family_status', 'family_status_id')

# вывод переменной 'family_stat_dict'
family_stat_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Столбцов `purpose_type` и `purpose_type_id` c переменной `purpose_dict`:

In [51]:
# вызов функции 'create_category_dict()' для столбцов 'purpose_type' и 'purpose_type_id'
purpose_dict = create_category_dict('purpose_type', 'purpose_type_id')

# вывод переменной 'purpose_dict'
purpose_dict

,purpose_type,purpose_type_id
0,недвижимость,0
1,автомобиль,1
2,образование,2
3,свадьба,3


Также создадим словарь для столбцов `income_type` и `income_type_id` c переменной `income_type_dict`:

In [52]:
# вызов функции 'create_category_dict()' для столбцов 'income_type' и 'income_type_id'
income_type_dict = create_category_dict('income_type', 'income_type_id')

# вывод переменной 'income_type_dict'
income_type_dict

,income_type,income_type_id
0,сотрудник,0
1,пенсионер,1
2,компаньон,2
3,госслужащий,3
4,безработный,4
5,предприниматель,5
6,студент,6
7,в декрете,7


И полей `total_income_type` и `total_income_type_id` c переменной `total_income_type_dict`:

In [53]:
# вызов функции 'create_category_dict()' для столбцов 'total_income_type' и 'total_income_type_id'
total_income_type_dict = create_category_dict('total_income_type', 'total_income_type_id')

# вывод переменной 'total_income_type_dict'
total_income_type_dict

,total_income_type,total_income_type_id
0,богатый,0
1,средний класс,1
2,малоимущий,2


А также `children_type` и `children_type_id` c переменной `children_type_dict`:

In [54]:
# вызов функции 'create_category_dict()' для столбцов 'children_type' и 'children_type_id'
children_type_dict = create_category_dict('children_type', 'children_type_id')

# вывод переменной 'children_type_dict'
children_type_dict

,children_type,children_type_id
0,малодетный,0
1,бездетный,1
2,многодетный,2


**Вывод**

Игнорирование лемматизации фактически создает помехи исследованию данных в столбцах со стороковым типом значений - проверка вхождения подстроки в строку в таком случае даст непредсказуемый и некорректный результат. Представление категорий в строковом типе в датасетах усложняет визуальное восприятие информации, а также увеличивает время обрабоки данных. Для решения данных проблем в этом разделе была проведена:
* лемматизация столбца `purpose` с выделением категорий в `purpose_type` и их id в `purpose_type_id`
* классификация:
    - по количеству детей `children_type` с помещением их id в столбец `children_type_id`
    - типам занятости `income_type` и id в столбец `income_type_id`
    - уровня дохода `total_income_type` с помещением их id в столбец `total_income_type_id`
* создание словарей *"значение"* -> *"id"* соответствующих категорий:
    - `edu_dict`
    - `family_stat_dict`
    - `purpose_dict`
    - `income_type_dict`
    - `total_income_type_dict`
    - `children_type_dict` 

## Проверка гипотез исследования

Для последующего анализа создадим функцию `create_pivot_analysis`, которая будет возвращать сводную таблицу с категориями и долями "должников" по каждой категории:

In [55]:
# создание функции 'create_pivot_analysis()' с аргументом 'category'
def create_pivot_analysis(category):
    '''
    Возвращает сводную таблицу с категориями
    и долями "должников" по каждой категории.
    '''
    pivot_category = data.pivot_table(index=[category], values= 'debt', aggfunc=['mean'])\
                         .sort_values(by=('mean', 'debt'), ascending=False).round(3)

    return pivot_category

**Вопрос I.** Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на этот вопрос вызовем функцию `create_pivot_analysis` с аргументом `children`. Получим долю клиентов, не вернувших кредит в срок, по категориям детности. Сохраним в переменной `pivot_children`:

In [56]:
# вызов функции 'create_pivot_analysis()' с аргументом 'children'
pivot_children = create_pivot_analysis('children_type')

# вывод переменной 'pivot_children'
pivot_children

,mean
,debt
children_type,
малодетный,0.093
многодетный,0.082
бездетный,0.075


Как видно из полученной таблицы, доли "должников" от общего числа клиентов по категориям разнятся.

**Вывод**

По итогу исследования наблюдаем, что зависимость между наличием детей и возвратом кредита прослеживается - доля клиентов, просрочивших обязательства по кредиту, с 1-3 детьми составляет 9%, у многодетных 8%, бездетных - 7.5%.

В результате исследования первая гипотеза подтверждена.

**Вопрос II.** Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгруппируем с помощью функции `create_pivot_analysis` сводную таблицу с долей клиентов, не вернувших кредит в срок, по семейному положению и сохраним ее в переменной `pivot_family_status`:

In [57]:
# вызов функции 'create_pivot_analysis()' с аргументом 'family_status'
pivot_family_status = create_pivot_analysis('family_status')

# вывод переменной 'pivot_family_status'
pivot_family_status

,mean
,debt
family_status,
не женат / не замужем,0.098
гражданский брак,0.093
женат / замужем,0.075
в разводе,0.071
вдовец / вдова,0.066


Наблюдаем отличия в долях "должников" по группам: самая большая - у клиентов, не состоящих в официальных отношениях, малая - у овдовевших и разведенных.

**Вывод**

Результаты показывают зависимость между семейным положением и возвратом кредита в срок - доля просрочек по кредитным обязательствам у клиентов, не состоящих в официальных отношениях больше всего и колеблется между 9 и 10%. Овдовевшие и разведенные клиенты чаще возвращают средства в срок, их доля от общего числа в группе - около 7%.

В результате исследования вторая гипотеза подтверждена.

**Вопрос III.** Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сформируем с помощью функции `create_pivot_analysis` долю клиентов с имевшейся задолженностью по кредитным обязательствам в группах по уровню дохода и сохраним в переменной `pivot_total_income_type`:

In [58]:
# вызов функции 'create_pivot_analysis()' с аргументом 'total_income_type'
pivot_total_income_type = create_pivot_analysis('total_income_type')

# вывод переменной 'pivot_total_income_type'
pivot_total_income_type

,mean
,debt
total_income_type,
средний класс,0.087
малоимущий,0.080
богатый,0.071


Видим, что доля просрочек по кредиту зависит от уровня дохода.

**Вывод**

Как видно из полученных значений, доля клиентов, вовремя не выполнивших кредитные обязательства, больше среди среднего класса и малоимущих, что довольно ожидаемо.

В результате исследования третья гипотеза подтверждена.

**Вопрос IV.** Как разные цели кредита влияют на его возврат в срок?

Объединим с помощью функции `create_pivot_analysis` сводную таблицу с долями клиентов, не вернувших кредит в срок, по целям кредитования. Сохраним в переменной `pivot_purpose_type`:

In [59]:
# вызов функции 'create_pivot_analysis()' с аргументом 'purpose_type'
pivot_purpose_type = create_pivot_analysis('purpose_type')

# вывод переменной 'pivot_purpose_type'
pivot_purpose_type

,mean
,debt
purpose_type,
автомобиль,0.094
образование,0.092
свадьба,0.080
недвижимость,0.072


Как видно из полученной таблицы, доли "должников" от общего числа клиентов по категориям отлчаются на 1-3%.

**Вывод**

Цели кредита также оказывают влияние на невыполнение обязательств в срок - доли просрочек по категориям несколько разнятся.

В результате исследования четвертая гипотеза подтверждена.

## Общий вывод исследования

**Рабочие гипотезы**
1. наличие детей влияет на возврат кредита в срок
2. возврат кредита в срок зависит от семейного положения заемщика
3. возврат кредита в срок зависит от уровня дохода заемщика
4. различные цели кредита влияют на его возврат в срок

По итогу исследования наблюдаем, что **зависимость между наличием детей и возвратом кредита прослеживается** - доля клиентов, просрочивших обязательства по кредиту, с одним-тремя детьми составляет 9%, у многодетных 8%, бездетных - 7.5%.

Скорее всего, это связано с дополнительными расходами на детей, и, соответственно, другим типом бюджета. У бездетных клиентов трата средств связана только с собственными нуждами и формируется исходя только из своих потребностей.

**Результаты показывают зависимость между семейным положением и возвратом кредита в срок** - доля просрочек по кредитным обязательствам у клиентов, не состоящих в официальных отношениях больше всего и колеблется между 9 и 10%. К этому приводит отсутствие совместного семейного бюджета, траты которого обсуждаются и контролируются обоими супругами. 

Играет роль также психологический фактор нахождения в браке и, как следствие, рациональность подхода к совместным тратам. Плюсом к этому дети рождаются зачастую именно в официальных браках, а это, как указывалось ранее, дополнительный стимулирующий фактор "осторожных" трат и финансовой дисциплины. 

Овдовевшие и разведенные клиенты в буквальном смысле остаются "одни", не могут рассчитывать на партнера и грамотнее планируют свой бюджет.  

Как видно из полученных значений, **доля клиентов, вовремя не выполнивших кредитные обязательства, больше среди среднего класса и малоимущих**, что довольно ожидаемо. Доля "должников" по категории малоимущих несколько ниже среднего класса, так как клиенты данного уровня заработка умеют строже планировать ограниченные средства и более неприхотливы в бытовом плане.

**Довольно весом вклад цели кредитования в возможность невыплаты займа вовремя**. При оформлении автокредитов, клиенты чаще руководствуются личными предпочтениями и реже - рациональностью покупки, при этом не обращая внимания на стоимость содержания того или иного авто, что со временем вносит заметные траты в бюджет. 

Кредиты на образование могут быть невозвращены вовремя в связи с долгим поиском работы после обучения, а также личными обстоятельствами по прекращению или приостановлению образования. 

Свадебные кредиты погашаются вовремя ввиду материальной поддержки родственников клиентов. 

Дорогие ипотечные кредиты выдаются на более продолжительный срок, чем по остальным категориям - ежемесячный платеж меньше, процентная ставка ниже. Также, клиенты, оформляющие кредиты на недвижимость, трезво оценивают свои силы и планируют расходы исходя из размера ежемесячного платежа, стараясь при этом больше работать или получать дополнительный доход на стороне для нивелирования трат, связанных со взятыми на себя кредитными обязательствами.

**Итог**
- первая гипотеза подтверждена
- вторая гипотеза подтверждена
- третья гипотеза подтверждена
- четвертая гипотеза подтверждена